In [2]:
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 6.5 MB/s eta 0:00:00


In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import MCPhaseGate, ZGate # Import ZGate
import numpy as np
import math

def sudoku_solver(grid):
    n = 2  # 4x4 Sudoku
    grid_qubits = 32  # 16 cells * 2 qubits/cell
    oracle_ancillas = 6  # all_valid, constraint_violated, flags[0..3]
    total_qubits = grid_qubits + oracle_ancillas
    qr = QuantumRegister(total_qubits, 'q')
    cr = ClassicalRegister(grid_qubits, 'c')
    qc = QuantumCircuit(qr, cr)

    # Initialize fixed and variable cells
    variable_cells = []
    for i in range(4):
        for j in range(4):
            idx = 2 * (4 * i + j)
            if grid[i][j] != 0:
                value = grid[i][j] - 1
                bin_rep = format(value, '02b')
                if bin_rep[0] == '1':
                    qc.x(qr[idx])
                if bin_rep[1] == '1':
                    qc.x(qr[idx+1])
            else:
                qc.h(qr[idx])
                qc.h(qr[idx+1])
                variable_cells.append((i, j))

    # Identify variable qubits
    variable_qubits = []
    for (i, j) in variable_cells:
        idx = 2 * (4 * i + j)
        variable_qubits.append(idx)
        variable_qubits.append(idx+1)
    num_var_qubits = len(variable_qubits)

    # Initialize all_valid qubit (index 32) to |1>
    all_valid_idx = 32
    qc.x(qr[all_valid_idx])

    # Define constraints (rows, columns, boxes)
    constraints = []
    for i in range(4):  # Rows
        constraints.append([(i, j) for j in range(4)])
    for j in range(4):  # Columns
        constraints.append([(i, j) for i in range(4)])
    boxes = [
        [(0,0), (0,1), (1,0), (1,1)],
        [(0,2), (0,3), (1,2), (1,3)],
        [(2,0), (2,1), (3,0), (3,1)],
        [(2,2), (2,3), (3,2), (3,3)]
    ]
    constraints.extend(boxes)

    # Ancilla indices
    constraint_violated_idx = 33
    flags_idxs = [34, 35, 36, 37]

    # Calculate number of iterations
    k = len(variable_cells)
    N = 4 ** k
    T = max(1, int(round((np.pi / 4) * np.sqrt(N))))

    # Define oracle
    def apply_oracle():
        # Oracle implementation remains the same
        for constraint in constraints:
            for (i, j) in constraint:
                idx = 2 * (4 * i + j)
                for flag_idx, ctrl_state in zip(flags_idxs, ['00', '01', '10', '11']):
                    # Ensure correct mapping of qubits to MCX
                    qc.mcx([qr[idx], qr[idx+1]], qr[flag_idx], ctrl_state=ctrl_state)

            # Check for violations
            qc.x(qr[constraint_violated_idx])
            # Control on all flag qubits being 1
            qc.mcx([qr[i] for i in flags_idxs], qr[constraint_violated_idx], ctrl_state='1111')
            # Flip all_valid if constraint_violated is 1
            qc.cx(qr[constraint_violated_idx], qr[all_valid_idx])
            # Uncompute constraint_violated flag
            qc.mcx([qr[i] for i in flags_idxs], qr[constraint_violated_idx], ctrl_state='1111')
            qc.x(qr[constraint_violated_idx])

            # Uncompute flag qubits
            for (i, j) in constraint:
                idx = 2 * (4 * i + j)
                for flag_idx, ctrl_state in zip(flags_idxs, ['00', '01', '10', '11']):
                    qc.mcx([qr[idx], qr[idx+1]], qr[flag_idx], ctrl_state=ctrl_state)

        # Apply phase to all_valid qubit if it is |1> (valid solution)
        qc.z(qr[all_valid_idx])

        # Uncompute violation checks and flags (inverse of the above)
        for constraint in reversed(constraints): # Uncompute in reverse order
             for (i, j) in constraint:
                idx = 2 * (4 * i + j)
                for flag_idx, ctrl_state in zip(flags_idxs, ['00', '01', '10', '11']):
                    # Uncompute flag qubits
                    qc.mcx([qr[idx], qr[idx+1]], qr[flag_idx], ctrl_state=ctrl_state)

             # Uncompute constraint_violated flag
             qc.x(qr[constraint_violated_idx])
             qc.mcx([qr[i] for i in flags_idxs], qr[constraint_violated_idx], ctrl_state='1111')
             # Flip all_valid if constraint_violated is 1
             qc.cx(qr[constraint_violated_idx], qr[all_valid_idx])
             # Uncompute constraint_violated flag
             qc.mcx([qr[i] for i in flags_idxs], qr[constraint_violated_idx], ctrl_state='1111')
             qc.x(qr[constraint_violated_idx])

    # Define diffuser
    def apply_diffuser():
        qc.h(variable_qubits)
        # Apply X gates to flip |0> to |1>
        qc.x(variable_qubits)
        # Apply a multi-controlled Z gate conditioned on all variable qubits

        mcz = MCPhaseGate(math.pi, num_var_qubits, ctrl_state='1'*num_var_qubits)

        variable_qr_list = [qr[i] for i in variable_qubits]
        # Reconstruct the gate and append
        mcz = MCPhaseGate(math.pi, num_var_qubits, ctrl_state='1'*num_var_qubits)


        # Apply X gates again to flip back
        qc.x(variable_qubits)
        qc.h(variable_qubits)

        # APPLYING THE CORRECT DIFFUSER IMPLEMENTATION
        qc.h(variable_qubits)
        # Apply X gates to flip |0> to |1> on variable qubits
        qc.x(variable_qubits)
        # Apply a multi-controlled Z gate on the last variable qubit, controlled by all others.
        # This creates a phase flip for the state |1...1> of all variable qubits.
        # Number of controls is num_var_qubits - 1.
        # The last qubit in variable_qubits is the target.
        num_controls = num_var_qubits - 1
        if num_controls >= 1: # Need at least one control for a controlled gate
            controlled_z = ZGate().control(num_controls, ctrl_state='1'*num_controls)
            control_qubits = [qr[i] for i in variable_qubits[:-1]]
            target_qubit = qr[variable_qubits[-1]]
            qc.append(controlled_z, control_qubits + [target_qubit])
        else: # If only one variable qubit, just apply a Z gate
             qc.z(qr[variable_qubits[0]])


        # Apply X gates again to flip back
        qc.x(variable_qubits)
        qc.h(variable_qubits)

    # Apply Grover iterations
    for _ in range(T):
        apply_oracle()
        apply_diffuser()

    # Measure grid qubits
    qc.measure([qr[i] for i in range(grid_qubits)], [cr[i] for i in range(grid_qubits)])

    return qc

# Example grid with 2 variable cells
grid_example = [
    [1, 0, 3, 4],
    [3, 4, 1, 2],
    [2, 1, 4, 3],
    [4, 3, 2, 0]
]

# Generate the circuit
circuit = sudoku_solver(grid_example)
print(circuit)

                                                                            »
 q_0: ───────o────■─────────o──────────────■────────────────────────────────»
             │    │         │              │                                »
 q_1: ───────o────o─────────■──────────────■────────────────────────────────»
      ┌───┐  │    │         │              │                                »
 q_2: ┤ H ├──┼────┼────o────┼────■─────────┼────o──────────────■────────────»
      ├───┤  │    │    │    │    │         │    │              │            »
 q_3: ┤ H ├──┼────┼────o────┼────o─────────┼────■──────────────■────────────»
      ├───┤  │    │    │    │    │         │    │              │            »
 q_4: ┤ X ├──┼────┼────┼────┼────┼────o────┼────┼────■─────────┼────o───────»
      └───┘  │    │    │    │    │    │    │    │    │         │    │       »
 q_5: ───────┼────┼────┼────┼────┼────o────┼────┼────o─────────┼────■───────»
      ┌───┐  │    │    │    │    │    │    │    │    │         │